<a href="https://colab.research.google.com/github/Baraa42/Odin-Recipe/blob/master/CIFAR10_ColabGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
import matplotlib.pyplot as plt
import glob,os
import matplotlib.image as mpimg
import torchvision
from torchvision.datasets import ImageFolder, CIFAR10


In [2]:
#Check if GPU is available 
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
print("Device:",device)

Device: cuda:0


# **Getting Data**

In [3]:
train_dataset = CIFAR10(
    root='.',
    train=True,
    transform=transforms.ToTensor(),
    download=True)

test_dataset = CIFAR10(
    root='.',
    train=False,
    transform=transforms.ToTensor(),
    download=True)



  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to .
Files already downloaded and verified


In [4]:
classes = train_dataset.classes
print(classes)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


# **Preparing Data and Functions**

In [15]:
from torch.utils.data import DataLoader
batch_size = 128
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=batch_size,
                          shuffle=False)

**Run CNN function**

In [16]:
def run_CNN(model, criterion, optimizer, train_loader, test_loader, epochs=10):
    '''
    Runs the model and returns the list of train/test losses/accuracies
    '''
    # Losses
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)
    train_accuracy = np.zeros(epochs)
    test_accuracy = np.zeros(epochs)
    
    for i in range(epochs):
        train_loss =[]
        correct = 0 
        total = 0
        for inputs, targets in train_loader :

            inputs, targets = inputs.to(device), targets.to(device)
            # Resetting gradient
            optimizer.zero_grad()
            # Getting output and Loss
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            # Back prop
            loss.backward()
            optimizer.step()
            # Train loss after one minibatch
            train_loss.append(loss.item())
            # Getting prediction
            _, predictions = torch.max(outputs, 1)
            # Getting number correct prediction
            correct += (predictions==targets).sum().item()
            total += targets.shape[0]
        # Train loss after epoch and accuracy
        train_loss = np.mean(train_loss)
        train_losses[i] = train_loss
        train_accuracy[i]= correct/total
        
        test_loss =[]
        correct = 0 
        total = 0
        for inputs, targets in test_loader:

            inputs, targets = inputs.to(device), targets.to(device)
            #Getting output and Loss
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            # Test loss after one minibatch
            test_loss.append(loss.item())
            # Getting prediction
            _, predictions = torch.max(outputs, 1)
            # Getting number correct prediction
            correct += (predictions==targets).sum().item()
            total += targets.shape[0]
        # Test loss after epoch  and accuracy 
        test_loss = np.mean(test_loss)
        test_losses[i] = test_loss 
        test_accuracy[i]= correct/total
         
        print(f'Epochs {i+1}/{epochs}, Train Loss: {train_loss:.3f}, Test Loss : {test_loss:.3f}, Train Accuracy: {train_accuracy[i]:.3f}, Test Accuracy : {test_accuracy[i]:.3f}')
        
    return train_losses, test_losses, train_accuracy, test_accuracy

# **Building Improved CNN**

In [28]:
K = 10
class CNN(nn.Module):
    def __init__(self, K):
        super(CNN, self).__init__()
        
        # conv layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=(1,1))
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=(1,1))
    
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=(1,1))
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=(1,1))
        
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, padding=(1,1))
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=(1,1))
        
        # liner layers
        self.fc1 = nn.Linear(128*4*4, 1024)
        self.fc2 = nn.Linear(1024, K)
        
        
    def forward(self, X):
        m = nn.MaxPool2d(2)
        out = F.relu(self.conv1(X))
        out = F.relu(self.conv2(out))
        out = m(out)
        out = F.relu(self.conv3(out))
        out = F.relu(self.conv4(out))
        out = m(out)
        out = F.relu(self.conv5(out))
        out = F.relu(self.conv6(out))
        out = m(out)
        out = out.view(-1, 128*4*4)
        out = F.dropout(out, p=0.2)
        out = F.relu(self.fc1(out))
        out = F.dropout(out, p=0.2)
        out = F.relu(self.fc2(out))
        return out

In [30]:
model = CNN(K).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [31]:
train_losses, test_losses, train_accuracy, test_accuracy = run_CNN(model, criterion, optimizer, train_loader, test_loader, epochs=15)

Epochs 1/15, Train Loss: 2.303, Test Loss : 2.302, Train Accuracy: 0.099, Test Accuracy : 0.107
Epochs 2/15, Train Loss: 2.300, Test Loss : 2.277, Train Accuracy: 0.124, Test Accuracy : 0.144
Epochs 3/15, Train Loss: 2.062, Test Loss : 1.869, Train Accuracy: 0.245, Test Accuracy : 0.316
Epochs 4/15, Train Loss: 1.751, Test Loss : 1.580, Train Accuracy: 0.361, Test Accuracy : 0.431
Epochs 5/15, Train Loss: 1.500, Test Loss : 1.434, Train Accuracy: 0.455, Test Accuracy : 0.487
Epochs 6/15, Train Loss: 1.355, Test Loss : 1.277, Train Accuracy: 0.514, Test Accuracy : 0.543
Epochs 7/15, Train Loss: 1.224, Test Loss : 1.187, Train Accuracy: 0.561, Test Accuracy : 0.576
Epochs 8/15, Train Loss: 1.095, Test Loss : 1.108, Train Accuracy: 0.612, Test Accuracy : 0.600
Epochs 9/15, Train Loss: 0.976, Test Loss : 0.991, Train Accuracy: 0.656, Test Accuracy : 0.645
Epochs 10/15, Train Loss: 0.874, Test Loss : 0.961, Train Accuracy: 0.695, Test Accuracy : 0.663
Epochs 11/15, Train Loss: 0.780, Test L

In [32]:
train_losses, test_losses, train_accuracy, test_accuracy = run_CNN(model, criterion, optimizer, train_loader, test_loader, epochs=10)

Epochs 1/10, Train Loss: 0.399, Test Loss : 0.982, Train Accuracy: 0.860, Test Accuracy : 0.698
Epochs 2/10, Train Loss: 0.343, Test Loss : 0.974, Train Accuracy: 0.879, Test Accuracy : 0.707
Epochs 3/10, Train Loss: 0.294, Test Loss : 1.080, Train Accuracy: 0.896, Test Accuracy : 0.705
Epochs 4/10, Train Loss: 0.259, Test Loss : 1.123, Train Accuracy: 0.909, Test Accuracy : 0.711
Epochs 5/10, Train Loss: 0.232, Test Loss : 1.143, Train Accuracy: 0.919, Test Accuracy : 0.707
Epochs 6/10, Train Loss: 0.204, Test Loss : 1.177, Train Accuracy: 0.928, Test Accuracy : 0.711
Epochs 7/10, Train Loss: 0.186, Test Loss : 1.196, Train Accuracy: 0.935, Test Accuracy : 0.711
Epochs 8/10, Train Loss: 0.161, Test Loss : 1.295, Train Accuracy: 0.945, Test Accuracy : 0.709
Epochs 9/10, Train Loss: 0.142, Test Loss : 1.394, Train Accuracy: 0.951, Test Accuracy : 0.707
Epochs 10/10, Train Loss: 0.150, Test Loss : 1.288, Train Accuracy: 0.949, Test Accuracy : 0.725
